---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [35]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [36]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [150]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open ('university_towns.txt') as file:
        data = []
        for a in file:
            data.append(a[: -1])
        
    state_town = []
    for a in data:
        if a[-6:] == '[edit]':
            state = a[: -6]
        elif '(' in a:
            town = a[:a.index('(')-1]
            state_town.append([state, town])
        else:
            town = a
            state_town.append([state, town])
    
    df = pd.DataFrame(state_town, columns=['State', 'RegionName'])

    return df

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


From Bureau of Economic Analysis, US Department of Commerce, the GDP over time of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file gdplev.xls. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [152]:
gdp = pd.read_excel('gdplev.xls',skiprows=7)
gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
gdp.columns = ['Quarter', 'GDP']
gdp = gdp[212:]
gdp[10:-10]

,Quarter,GDP
222,2002q3,11037.1
223,2002q4,11103.8
224,2003q1,11230.1
225,2003q2,11370.7
226,2003q3,11625.1
227,2003q4,11816.8
228,2004q1,11988.4
229,2004q2,12181.4
230,2004q3,12367.7
231,2004q4,12562.2


In [153]:
gdp.loc[212+2, 'GDP']

10357.4

In [154]:
starts = []
for i in gdp.index[:-2]:
    
    if (gdp.loc[i, 'GDP'] > gdp.loc[i+1, 'GDP']) & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']):
        starts.append(gdp.loc[i, 'Quarter'])
        
starts[0]

'2008q3'

In [155]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp.columns = ['Quarter', 'GDP']
    gdp = gdp[212:]
    
    starts = []
    for i in gdp.index[:-2]:
        if (gdp.loc[i, 'GDP'] > gdp.loc[i+1, 'GDP']) & (gdp.loc[i+1, 'GDP'] > gdp.loc[i+2, 'GDP']):
            starts.append(gdp.loc[i, 'Quarter'])
    
    return starts[0]

get_recession_start()

'2008q3'

In [156]:
gdp[gdp['Quarter'] > '2008q3'].index

Int64Index([247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
            260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272,
            273, 274, 275, 276, 277],
           dtype='int64')

In [157]:
gdp[gdp['Quarter'] > '2008q3'].index[3:]

Int64Index([250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262,
            263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275,
            276, 277],
           dtype='int64')

In [158]:
ends = []
for i in gdp[gdp['Quarter'] > '2008q3'].index[3:]:
    if (gdp.loc[i, 'GDP'] > gdp.loc[i-1, 'GDP']) & (gdp.loc[i-1, 'GDP'] > gdp.loc[i-2, 'GDP']):
        ends.append(gdp.loc[i, 'Quarter'])
ends[0]

'2009q4'

In [159]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp.columns = ['Quarter', 'GDP']
    gdp = gdp[212:]
    
    ends = []
    for i in gdp[gdp['Quarter'] > '2008q3'].index[3:]:
        if (gdp.loc[i, 'GDP'] > gdp.loc[i-1, 'GDP']) & (gdp.loc[i-1, 'GDP'] > gdp.loc[i-2, 'GDP']):
            ends.append(gdp.loc[i, 'Quarter'])
   
    return ends[0]

get_recession_end()

'2009q4'

In [160]:
gdp[(gdp['Quarter'] >= '2008q3') & (gdp['Quarter'] <= '2009q4')].index

Int64Index([246, 247, 248, 249, 250, 251], dtype='int64')

In [161]:
recession = gdp[(gdp['Quarter'] >= '2008q3') & (gdp['Quarter'] <= '2009q4')]
recession

,Quarter,GDP
246,2008q3,14843.0
247,2008q4,14549.9
248,2009q1,14383.9
249,2009q2,14340.4
250,2009q3,14384.1
251,2009q4,14566.5


In [162]:
min(recession['GDP'])

14340.4

In [163]:
bottom = recession[recession['GDP'] == min(recession['GDP'])]
bottom

,Quarter,GDP
249,2009q2,14340.4


In [164]:
bottom['Quarter'].tolist()[0]

'2009q2'

In [165]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp.columns = ['Quarter', 'GDP']
    gdp = gdp[212:]
    
    recession = gdp[(gdp['Quarter'] >= '2008q3') & (gdp['Quarter'] <= '2009q4')]
    bottom = recession[recession['GDP'] == min(recession['GDP'])]
    
    return bottom['Quarter'].tolist()[0]

get_recession_bottom()

'2009q2'

In [166]:
housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
housing_df = housing_df.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1)
housing_df.head()

,RegionName,State,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,CA,155000.0,154600.0,154400.0,154200.0,154100.0,154300.0,154300.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,Chicago,IL,109700.0,109400.0,109300.0,109300.0,109100.0,109000.0,109000.0,109600.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,Philadelphia,PA,50000.0,49900.0,49600.0,49400.0,49400.0,49300.0,49300.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,Phoenix,AZ,87200.0,87700.0,88200.0,88400.0,88500.0,88900.0,89400.0,89700.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [167]:
# clean up the DataFrame with only the info we need

col_to_drop = [col for col in housing_df.columns[2:] if col < '2000-01']
housing_df = housing_df.drop(col_to_drop, axis=1)
housing_df.head()

,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,CA,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,Chicago,IL,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,Philadelphia,PA,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,Phoenix,AZ,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [168]:
housing_df.columns[2:-2]

Index(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06',
       '2000-07', '2000-08', '2000-09', '2000-10',
       ...
       '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02',
       '2016-03', '2016-04', '2016-05', '2016-06'],
      dtype='object', length=198)

In [169]:
housing_df = housing_df.sort(columns='State')
housing_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
3697,Kenai,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,206200,207600,208800,209800,210300,211200,211800,212000,212100,211800
5150,Ketchikan,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,196700,196600,195700,195200,195900,201000,206700,208200,206500,206400
10081,Anchor Point,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,154700,155800,156300,156200,157900,161200,162300,161100,161000,161400
55,Anchorage,AK,175700.0,174700.0,173500.0,173800.0,175300.0,176700.0,178000.0,179500.0,...,291600,292100,293100,293900,294100,294500,295000,295000,294800,294400
1770,Palmer,AK,169800.0,168800.0,167700.0,168500.0,170700.0,172500.0,173300.0,174400.0,...,241300,242900,245500,247000,246700,246200,245700,245300,245000,244500


In [170]:
housing_df.set_index(['State', 'RegionName'], inplace=True)

In [171]:
housing_df.head()

2000-01   2000-02   2000-03   2000-04   2000-05  \
State RegionName                                                       
AK    Kenai              NaN       NaN       NaN       NaN       NaN   
      Ketchikan          NaN       NaN       NaN       NaN       NaN   
      Anchor Point       NaN       NaN       NaN       NaN       NaN   
      Anchorage     175700.0  174700.0  173500.0  173800.0  175300.0   
      Palmer        169800.0  168800.0  167700.0  168500.0  170700.0   

                     2000-06   2000-07   2000-08   2000-09   2000-10   ...     \
State RegionName                                                       ...      
AK    Kenai              NaN       NaN       NaN       NaN       NaN   ...      
      Ketchikan          NaN       NaN       NaN       NaN       NaN   ...      
      Anchor Point       NaN       NaN       NaN       NaN       NaN   ...      
      Anchorage     176700.0  178000.0  179500.0  181200.0  182500.0   ...      
      Palmer        172500.0  173300.0  174400.0  175000.0  175200.0   ...      

                    2015-11  2015-12  2016-01  2016-02  2016-03  2016-04  \
State RegionName                                                           
AK    Kenai          206200   207600   208800   209800   210300   211200   
      Ketchikan      196700   196600   195700   195200   195900   201000   
      Anchor Point   154700   155800   156300   156200   157900   161200   
      Anchorage      291600   292100   293100   293900   294100   294500   
      Palmer         241300   242900   245500   247000   246700   246200   

                    2016-05  2016-06  2016-07  2016-08  
State RegionName                                        
AK    Kenai          211800   212000   212100   211800  
      Ketchikan      206700   208200   206500   206400  
      Anchor Point   162300   161100   161000   161400  
      Anchorage      295000   295000   294800   294400  
      Palmer         245700   245300   245000   244500  

[5 rows x 200 columns]

In [172]:
np.arange(len(housing_df.columns)) // 3

array([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,
        5,  6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11,
       11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16,
       17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22,
       22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28,
       28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33,
       34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39,
       39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45,
       45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 50, 50, 50,
       51, 51, 51, 52, 52, 52, 53, 53, 53, 54, 54, 54, 55, 55, 55, 56, 56,
       56, 57, 57, 57, 58, 58, 58, 59, 59, 59, 60, 60, 60, 61, 61, 61, 62,
       62, 62, 63, 63, 63, 64, 64, 64, 65, 65, 65, 66, 66])

In [173]:
housing_quarters_df = housing_df.groupby((np.arange(len(housing_df.columns)) // 3), axis=1).sum()
housing_quarters_df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66],
           dtype='int64')

In [174]:
col_names = ['200%dq%d' % (int(x/4), x%4+1) for x in housing_quarters_df.columns]
col_names

['2000q1',
 '2000q2',
 '2000q3',
 '2000q4',
 '2001q1',
 '2001q2',
 '2001q3',
 '2001q4',
 '2002q1',
 '2002q2',
 '2002q3',
 '2002q4',
 '2003q1',
 '2003q2',
 '2003q3',
 '2003q4',
 '2004q1',
 '2004q2',
 '2004q3',
 '2004q4',
 '2005q1',
 '2005q2',
 '2005q3',
 '2005q4',
 '2006q1',
 '2006q2',
 '2006q3',
 '2006q4',
 '2007q1',
 '2007q2',
 '2007q3',
 '2007q4',
 '2008q1',
 '2008q2',
 '2008q3',
 '2008q4',
 '2009q1',
 '2009q2',
 '2009q3',
 '2009q4',
 '20010q1',
 '20010q2',
 '20010q3',
 '20010q4',
 '20011q1',
 '20011q2',
 '20011q3',
 '20011q4',
 '20012q1',
 '20012q2',
 '20012q3',
 '20012q4',
 '20013q1',
 '20013q2',
 '20013q3',
 '20013q4',
 '20014q1',
 '20014q2',
 '20014q3',
 '20014q4',
 '20015q1',
 '20015q2',
 '20015q3',
 '20015q4',
 '20016q1',
 '20016q2',
 '20016q3']

In [175]:
housing_quarters_df.columns = col_names
housing_quarters_df

2000q1    2000q2    2000q3    2000q4    2001q1  \
State RegionName                                                           
AK    Kenai                  NaN       NaN       NaN       NaN       NaN   
      Ketchikan              NaN       NaN       NaN       NaN       NaN   
      Anchor Point           NaN       NaN       NaN       NaN       NaN   
      Anchorage         523900.0  525800.0  538700.0  548500.0  548300.0   
      Palmer            506300.0  511700.0  522700.0  526600.0  508400.0   
      Seward                 NaN       NaN       NaN       NaN       NaN   
      Kodiak                 NaN       NaN       NaN       NaN       NaN   
      Tanaina           481500.0  490500.0  499100.0  504100.0  476300.0   
      Lakes             518200.0  522400.0  531800.0  537400.0  517200.0   
      North Pole        472400.0  477300.0  486400.0  496500.0  478300.0   
      Homer                  NaN       NaN       NaN       NaN       NaN   
      Soldotna               NaN       NaN       NaN       NaN       NaN   
      Juneau            577400.0  582900.0  585500.0  582500.0  591500.0   
      Fairbanks         489600.0  495100.0  507900.0  518400.0  493300.0   
AL    Birmingham        162100.0  163200.0  164900.0  168200.0  170500.0   
      Gurley                 NaN       NaN       NaN       NaN       NaN   
      Fosters           322500.0  330600.0  334200.0  347100.0  342600.0   
      Hope Hull         335000.0  335400.0  338900.0  343900.0  345800.0   
      Lipscomb               NaN       NaN       NaN       NaN       NaN   
      Guntersville      278800.0  280500.0  280800.0  281900.0  282400.0   
      Meadowbrook       601500.0  609000.0  620400.0  631500.0  636100.0   
      Northport         377600.0  379100.0  379900.0  380900.0  386000.0   
      Morris            372200.0  373500.0  372700.0  377400.0  376700.0   
      Good Hope              NaN       NaN       NaN       NaN       NaN   
      Orange Beach      664900.0  666400.0  672100.0  687700.0  708900.0   
      Robertsdale       260500.0  263000.0  267000.0  273300.0  283100.0   
      Cottondale        302500.0  307900.0  314300.0  315900.0  320200.0   
      Titus                  NaN       NaN       NaN       NaN       NaN   
      Leeds             281500.0  283300.0  285200.0  289100.0  296200.0   
      Homewood          475000.0  479000.0  483300.0  494800.0  503400.0   
...                          ...       ...       ...       ...       ...   
WI    Erin              625700.0  627300.0  629000.0  642400.0  677100.0   
WV    Nitro                  NaN       NaN       NaN       NaN       NaN   
      Eskdale                NaN       NaN       NaN       NaN       NaN   
      Mineral Wells     226600.0  231700.0  245900.0  255900.0  260100.0   
      Cross Lanes       234400.0  234200.0  237700.0  238400.0  236400.0   
      Dunbar                 NaN       NaN       NaN       NaN       NaN   
      Vienna            216800.0  223600.0  226000.0  229600.0  232200.0   
      Saint Albans      191500.0  194100.0  195600.0  197700.0  199100.0   
      South Charleston       NaN       NaN       NaN       NaN       NaN   
      Parkersburg       164700.0  167500.0  169900.0  170500.0  172200.0   
      Chesapeake             NaN       NaN       NaN       NaN       NaN   
      East Bank              NaN       NaN       NaN       NaN       NaN   
      Belle             153200.0  153900.0  156600.0  153500.0  153800.0   
      Marmet                 NaN       NaN       NaN       NaN       NaN   
      Upper Falls       159100.0  157000.0  156300.0  152700.0  151100.0   
      Waverly           187100.0  188800.0  195900.0  199800.0  203800.0   
      Sissonville       171600.0  174600.0  183600.0  182100.0  177000.0   
      Charleston        178600.0  178100.0  178100.0  178900.0  181200.0   
      Clendenin         119900.0  114600.0  121500.0  117000.0  128100.0   
      Washington        251700.0  258100.0  265000.0  271400.0  280500.0  

In [176]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_df = housing_df.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1)
    col_to_drop = [col for col in housing_df.columns[2:] if col < '2000-01']
    housing_df = housing_df.drop(col_to_drop, axis=1)
    housing_df = housing_df.sort(columns='State')
    housing_df.set_index(['State', 'RegionName'], inplace=True)
    
    housing_quarters_df = housing_df.groupby((np.arange(len(housing_df.columns)) // 3), axis=1).sum()
    housing_quarters_df.columns = ['200%dq%d' % (int(x/4), x%4+1) for x in housing_quarters_df.columns]
    
    return housing_quarters_df

convert_housing_data_to_quarters()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


2000q1    2000q2    2000q3    2000q4    2001q1  \
State RegionName                                                           
AK    Kenai                  NaN       NaN       NaN       NaN       NaN   
      Ketchikan              NaN       NaN       NaN       NaN       NaN   
      Anchor Point           NaN       NaN       NaN       NaN       NaN   
      Anchorage         523900.0  525800.0  538700.0  548500.0  548300.0   
      Palmer            506300.0  511700.0  522700.0  526600.0  508400.0   
      Seward                 NaN       NaN       NaN       NaN       NaN   
      Kodiak                 NaN       NaN       NaN       NaN       NaN   
      Tanaina           481500.0  490500.0  499100.0  504100.0  476300.0   
      Lakes             518200.0  522400.0  531800.0  537400.0  517200.0   
      North Pole        472400.0  477300.0  486400.0  496500.0  478300.0   
      Homer                  NaN       NaN       NaN       NaN       NaN   
      Soldotna               NaN       NaN       NaN       NaN       NaN   
      Juneau            577400.0  582900.0  585500.0  582500.0  591500.0   
      Fairbanks         489600.0  495100.0  507900.0  518400.0  493300.0   
AL    Birmingham        162100.0  163200.0  164900.0  168200.0  170500.0   
      Gurley                 NaN       NaN       NaN       NaN       NaN   
      Fosters           322500.0  330600.0  334200.0  347100.0  342600.0   
      Hope Hull         335000.0  335400.0  338900.0  343900.0  345800.0   
      Lipscomb               NaN       NaN       NaN       NaN       NaN   
      Guntersville      278800.0  280500.0  280800.0  281900.0  282400.0   
      Meadowbrook       601500.0  609000.0  620400.0  631500.0  636100.0   
      Northport         377600.0  379100.0  379900.0  380900.0  386000.0   
      Morris            372200.0  373500.0  372700.0  377400.0  376700.0   
      Good Hope              NaN       NaN       NaN       NaN       NaN   
      Orange Beach      664900.0  666400.0  672100.0  687700.0  708900.0   
      Robertsdale       260500.0  263000.0  267000.0  273300.0  283100.0   
      Cottondale        302500.0  307900.0  314300.0  315900.0  320200.0   
      Titus                  NaN       NaN       NaN       NaN       NaN   
      Leeds             281500.0  283300.0  285200.0  289100.0  296200.0   
      Homewood          475000.0  479000.0  483300.0  494800.0  503400.0   
...                          ...       ...       ...       ...       ...   
WI    Erin              625700.0  627300.0  629000.0  642400.0  677100.0   
WV    Nitro                  NaN       NaN       NaN       NaN       NaN   
      Eskdale                NaN       NaN       NaN       NaN       NaN   
      Mineral Wells     226600.0  231700.0  245900.0  255900.0  260100.0   
      Cross Lanes       234400.0  234200.0  237700.0  238400.0  236400.0   
      Dunbar                 NaN       NaN       NaN       NaN       NaN   
      Vienna            216800.0  223600.0  226000.0  229600.0  232200.0   
      Saint Albans      191500.0  194100.0  195600.0  197700.0  199100.0   
      South Charleston       NaN       NaN       NaN       NaN       NaN   
      Parkersburg       164700.0  167500.0  169900.0  170500.0  172200.0   
      Chesapeake             NaN       NaN       NaN       NaN       NaN   
      East Bank              NaN       NaN       NaN       NaN       NaN   
      Belle             153200.0  153900.0  156600.0  153500.0  153800.0   
      Marmet                 NaN       NaN       NaN       NaN       NaN   
      Upper Falls       159100.0  157000.0  156300.0  152700.0  151100.0   
      Waverly           187100.0  188800.0  195900.0  199800.0  203800.0   
      Sissonville       171600.0  174600.0  183600.0  182100.0  177000.0   
      Charleston        178600.0  178100.0  178100.0  178900.0  181200.0   
      Clendenin         119900.0  114600.0  121500.0  117000.0  128100.0   
      Washington        251700.0  258100.0  265000.0  271400.0  280500.0  

In [210]:
data = convert_housing_data_to_quarters()
data = data.loc[:, '2008q3':'2009q2']
data = data.reset_index()
data['State'] = data['State'].replace(states)
data.head()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,State,RegionName,2008q3,2008q4,2009q1,2009q2
0,Alaska,Kenai,NaN,NaN,NaN,NaN
1,Alaska,Ketchikan,NaN,NaN,NaN,NaN
2,Alaska,Anchor Point,NaN,NaN,NaN,NaN
3,Alaska,Anchorage,888500.0,862600.0,842500.0,815800.0
4,Alaska,Palmer,778400.0,775900.0,788900.0,791400.0


Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

In [179]:
data['price_ratio'] = (data['2008q3'] - data['2009q2']) / data['2009q2']
data.head()

,State,RegionName,2008q3,2008q4,2009q1,2009q2,price_ratio
0,Alaska,Kenai,NaN,NaN,NaN,NaN,NaN
1,Alaska,Ketchikan,NaN,NaN,NaN,NaN,NaN
2,Alaska,Anchor Point,NaN,NaN,NaN,NaN,NaN
3,Alaska,Anchorage,888500.0,862600.0,842500.0,815800.0,0.089115
4,Alaska,Palmer,778400.0,775900.0,788900.0,791400.0,-0.016427


In [180]:
uni_town = get_list_of_university_towns()
uni_town.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [183]:
uni_town_set = set()
for i, row in uni_town.iterrows():
    uni_town_set.add((row['State'], row['RegionName']))

def is_uni_town(row):
    town_str = (row['State'], row['RegionName'])
    if town_str in uni_town_set:
        return 1
    else:
        return 0

data['is_uni_town'] = data.apply(is_uni_town, axis=1)

In [184]:
data['is_uni_town']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       1
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
10700    0
10701    0
10702    0
10703    0
10704    0
10705    0
10706    0
10707    0
10708    0
10709    0
10710    0
10711    0
10712    0
10713    0
10714    0
10715    0
10716    0
10717    0
10718    0
10719    0
10720    0
10721    0
10722    0
10723    0
10724    0
10725    0
10726    0
10727    0
10728    0
10729    0
Name: is_uni_town, dtype: int64

In [198]:
is_uni_df = data[data['is_uni_town'] == 1].loc[:, 'price_ratio'].dropna()
not_uni_df = data[data['is_uni_town'] == 0].loc[:, 'price_ratio'].dropna()

In [215]:
from scipy import stats
p = stats.ttest_ind(is_uni_df, not_uni_df)[1]
p

0.0059324820207087431

In [208]:
def difference():
    if p<0.01:
        return True
    else:
        return False
difference()

True

In [207]:
is_uni_df.mean()

0.03642809822176276

In [209]:
def better():
    if is_uni_df.mean() > not_uni_df.mean():
        return 'non-university town'
    else:
        return 'university town'
better()    

'university town'

In [213]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where 
    different=True if the t-test is True at a p<0.01 (we reject the null hypothesis), 
    or different=False if otherwise (we cannot reject the null hypothesis). 
    
    The variable p should be equal to the exact p value returned from scipy.stats.ttest_ind(). 
    
    The value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    data = convert_housing_data_to_quarters()
    data = data.loc[:, '2008q3':'2009q2']
    data = data.reset_index()
    data['price_ratio'] = (data['2008q3'] - data['2009q2']) / data['2009q2'] # create a new colunm "price_ratio"
    data['State'] = data['State'].replace(states) # replace state short names, such as "Ak", to full name "Alaska"
    
    # create a new col, data['is_uni_town'], to check whether it is "university town" or not:
    uni_town = get_list_of_university_towns()
    
    uni_town_set = set()
    for i, row in uni_town.iterrows():
        uni_town_set.add((row['State'], row['RegionName']))

    def is_uni_town(row):
        town_str = (row['State'], row['RegionName'])
        if town_str in uni_town_set:
            return 1
        else:
            return 0

    data['is_uni_town'] = data.apply(is_uni_town, axis=1)
    
    is_uni_df = data[data['is_uni_town'] == 1].loc[:, 'price_ratio'].dropna()
    not_uni_df = data[data['is_uni_town'] == 0].loc[:, 'price_ratio'].dropna()
    
    from scipy import stats
    p = stats.ttest_ind(is_uni_df, not_uni_df)[1]
    
    def difference():
        if p < 0.01:
            return True
        else:
            return False
    
    def better():
        if is_uni_df.mean() > not_uni_df.mean():
            return 'non-university town'
        else:
            return 'university town'
    
    return (difference(), p, better())

run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(True, 0.0059324820207087431, 'university town')